In [ ]:
%matplotlib inline
from IPython.display import display, Markdown
from datetime import time, datetime, date, timedelta
import snakemd
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.dates as dates

import fitfile
from garmindb import ConfigManager
from garmindb.garmindb import GarminDb, Attributes, ActivitiesDb, Activities, StepsActivities, ActivityLaps, ActivityRecords
from idbutils.list_and_dict import list_not_none

from jupyter_funcs import format_number

db_params_dict = ConfigManager.get_db_params()

garmin_db = GarminDb(db_params_dict)
garmin_act_db = ActivitiesDb(db_params_dict)
measurement_system = Attributes.measurements_type(garmin_db)
unit_strings = fitfile.units.unit_strings[measurement_system]
distance_units = {"kilometers": "km"}[unit_strings[fitfile.units.UnitTypes.distance_long]]

def __format_activity(activity):
    if activity:
        if activity.is_steps_activity():
            steps_activity = StepsActivities.get(garmin_act_db, activity.activity_id)
#             return [activity.activity_id, activity.name, activity.start_time.strftime("%y%m%d"), activity.sport, format_number(activity.distance, 1), activity.elapsed_time, format_number(activity.avg_speed, 1), steps_activity.avg_pace, format_number(activity.calories)]
            return [activity.activity_id, activity.name, activity.start_time.strftime("%y%m%d"), activity.sport, format_number(activity.distance, 1), activity.elapsed_time, activity.moving_time, format_number(activity.avg_speed, 1), format_number(activity.calories)]
        return [activity.activity_id, activity.name, activity.start_time.strftime("%y%m%d"), activity.sport, format_number(activity.distance, 1), activity.elapsed_time, activity.moving_time, format_number(activity.avg_speed, 1), format_number(activity.calories)]
    return ['', '', '', '', '', '', '', '', '']


activities = Activities.get_latest(garmin_act_db, Activities.row_count(garmin_act_db))

In [ ]:
# All Actitivies

doc = snakemd.new_doc("activities")
rows = [__format_activity(activity) for activity in activities]
doc.add_header("All Recorded Activities", 3)
doc.add_table(['Id', 'Name', 'Date', 'Sport', f'Dist ({distance_units})', 'Elapsed Time', f'Moving Time', f'Speed ({unit_strings[fitfile.units.UnitTypes.speed]})', 'Calories'], rows)

display(Markdown(str(doc)))

In [ ]:
# Swimming Only

doc = snakemd.new_doc("swims")
rows = [__format_activity(activity) for activity in activities if ((activity.name is not None) and ("swim" in activity.name.lower()))]
doc.add_header("All Swims: {}".format(len(rows)), 3)
doc.add_table(['Id', 'Name', 'Date', 'Sport', f'Dist ({distance_units})', 'Elapsed Time', f'Moving Time', f'Speed ({unit_strings[fitfile.units.UnitTypes.speed]})', 'Calories'], rows)

display(Markdown(str(doc)))

In [ ]:
start_dates = []
moving_times = []
distances = []
time100m = []
for activity in activities:
    if ((activity.name is not None) and ("swim" in activity.name.lower())):
        s = activity.distance
        t = (datetime.combine(date.min, activity.moving_time) - datetime.min).total_seconds()
        
        if (t/s/10 < 150) and (s > 0.8):
            time100m += [t/s/10]
            start_dates += [activity.start_time]
            moving_times += [t]
            distances += [s]


fig = plt.figure(figsize=(12,5), dpi= 100, facecolor='w', edgecolor='k')
plt.scatter(start_dates, time100m, c=distances, vmin=0.8, cmap="rainbow")

x_num = dates.date2num(start_dates)
trend = np.polyfit(x_num, time100m, 1)
fit = np.poly1d(trend)
x_fit = np.linspace(x_num.min(), x_num.max())
plt.plot(dates.num2date(x_fit), fit(x_fit), "r--")

ax = plt.gca()
plt.grid(visible=True, which='major', color='#666666', linestyle='-')
plt.grid(visible=True, which='minor', color='#999999', linestyle='-', alpha=0.2)
years = plt.matplotlib.dates.YearLocator()
months = plt.matplotlib.dates.MonthLocator()
# yearsFmt = plt.matplotlib.dates.DateFormatter('%Y')
ax.xaxis.set_major_locator(years)
ax.xaxis.set_minor_locator(months)
# ax.xaxis.set_major_formatter(yearsFmt)

plt.title(f"Swim Time\nRecorded Swims: {len(rows)}")
plt.colorbar(label="Total Distance / km")
plt.ylabel("Time per 100m (s)")
plt.ylim(min(time100m), 140)
plt.show()

In [ ]:
# First 1km time

start_dates = []
moving_times = []
distances = []
time100m = []
for activity in activities:
    if ((activity.name is not None) and ("swim" in activity.name.lower())):
#         s = activity.distance
#         t = (datetime.combine(date.min, activity.moving_time) - datetime.min).total_seconds()
        
        laps = ActivityLaps.get_activity(garmin_act_db, activity.activity_id)
        s = 0
        t = 0
        for lap in laps:
            if (s < 1.0) and (lap.distance > 0.02):
                s += lap.distance
                t += (datetime.combine(date.min, lap.moving_time) - datetime.min).total_seconds()
        
        if (t/s/10 < 150) and (s > 0.8):
            time100m += [t/s/10]
            start_dates += [activity.start_time]
            moving_times += [t]
            distances += [s]


fig = plt.figure(figsize=(12,5), dpi= 100, facecolor='w', edgecolor='k')
plt.scatter(start_dates, time100m, c=distances, vmin=0.8, cmap="rainbow")

x_num = dates.date2num(start_dates)
trend = np.polyfit(x_num, time100m, 1)
fit = np.poly1d(trend)
x_fit = np.linspace(x_num.min(), x_num.max())
plt.plot(dates.num2date(x_fit), fit(x_fit), "r--")

ax = plt.gca()
plt.grid(visible=True, which='major', color='#666666', linestyle='-')
plt.grid(visible=True, which='minor', color='#999999', linestyle='-', alpha=0.2)
years = plt.matplotlib.dates.YearLocator()
months = plt.matplotlib.dates.MonthLocator()
# yearsFmt = plt.matplotlib.dates.DateFormatter('%Y')
ax.xaxis.set_major_locator(years)
ax.xaxis.set_minor_locator(months)
# ax.xaxis.set_major_formatter(yearsFmt)

plt.title(f"Swim Time - laps until exceeding 1km\nRecorded Swims: {len(rows)}")
plt.colorbar(label="Total Distance / km")
plt.ylabel("Time per 100m (s)")
plt.ylim(min(time100m), 140)
plt.show()

In [ ]:
# Running

start_dates = []
moving_times = []
distances = []
time1000m = []
for activity in activities:
    if ((activity.name is not None) and ("run" in activity.name.lower())):
        s = activity.distance
        t = (datetime.combine(date.min, activity.moving_time) - datetime.min).total_seconds()
        
        if (210 < t/s < 480) and (s > 0.5):
            time1000m += [t/s]
            start_dates += [activity.start_time]
            moving_times += [t]
            distances += [s]


fig = plt.figure(figsize=(12,5), dpi= 100, facecolor='w', edgecolor='k')
plt.scatter(start_dates, time1000m, c=distances, norm=matplotlib.colors.LogNorm(), cmap="rainbow")

x_num = dates.date2num(start_dates)
trend = np.polyfit(x_num, time1000m, 1)
fit = np.poly1d(trend)
x_fit = np.linspace(x_num.min(), x_num.max())
plt.plot(dates.num2date(x_fit), fit(x_fit), "r--")

ax = plt.gca()
plt.grid(visible=True, which='major', color='#666666', linestyle='-')
plt.grid(visible=True, which='minor', color='#999999', linestyle='-', alpha=0.2)
years = plt.matplotlib.dates.YearLocator()
months = plt.matplotlib.dates.MonthLocator()
# yearsFmt = plt.matplotlib.dates.DateFormatter('%Y')
ax.xaxis.set_major_locator(years)
ax.xaxis.set_minor_locator(months)
# ax.xaxis.set_major_formatter(yearsFmt)

plt.title("Run Time")
plt.colorbar(label="Total Distance / km")
plt.ylabel("Time per km (s)")
plt.ylim(240, 480)
plt.show()